# Challenge Exercise
Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.

In [24]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from math import pi
import shutil

print(tf.__version__)

1.8.0


The training dataset will be randomly generated. This should be a generator, but to keep it simple i'll generate it directly on RAM. I alsowant to learn how to work with panda dataframes.

In [25]:
COLUMNS=['radius', 'height', 'volume']

def generate_dataset(min_value, max_value, dataset_lenght):
  dataset = []
  for i in range(dataset_lenght):
    radius = random.uniform(min_value, max_value)
    height = random.uniform(min_value, max_value)
    volume = pi * radius**2 * height
    
    dataset.append((radius, height, volume))
  return dataset
    

df_train = pd.DataFrame(generate_dataset(0.5, 2.0, 100000), columns=COLUMNS)
df_valid = pd.DataFrame(generate_dataset(0.5, 2.0, 20000), columns=COLUMNS)

train.head()

,radius,height,volume
0,1.046283,1.189658,4.091386
1,0.769270,0.568194,1.056340
2,1.515097,0.756115,5.452790
3,1.010015,1.551065,4.970904
4,0.629675,1.844631,2.297692


In [36]:
LABEL=COLUMNS[2]
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

def make_prediction_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    shuffle = False
  )

FEATURES=COLUMNS[0:2]
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

In [27]:
def print_rmse(model, name, df):
  metrics = model.evaluate(input_fn = make_input_fn(df, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))

In [33]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_input_fn(df_train, num_epochs = 10));
print_rmse(model, 'validation', df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb188ccec10>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'taxi_trained', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 9901.495, ste

INFO:tensorflow:loss = 2.4142647, step = 7401 (0.237 sec)
INFO:tensorflow:global_step/sec: 388.801
INFO:tensorflow:loss = 2.5640826, step = 7501 (0.257 sec)
INFO:tensorflow:global_step/sec: 322.388
INFO:tensorflow:loss = 2.1890771, step = 7601 (0.310 sec)
INFO:tensorflow:global_step/sec: 408.433
INFO:tensorflow:loss = 2.726472, step = 7701 (0.245 sec)
INFO:tensorflow:global_step/sec: 425.414
INFO:tensorflow:loss = 2.2686625, step = 7801 (0.235 sec)
INFO:tensorflow:Saving checkpoints for 7813 into taxi_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 1.0253594.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-22-11:57:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-7813
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-22-11:57:41
INFO:tensorflow:Saving dict for global step

In [37]:
pcount=5
predictions = model.predict(input_fn = make_prediction_input_fn(df_valid[:pcount], 1))
print df_valid[0:pcount]
for i in xrange(pcount):
  print(predictions.next())
  
  

     radius    height     volume
0  1.699392  1.216494  11.036893
1  1.205358  0.734145   3.350922
2  1.853363  0.869212   9.379861
3  1.178952  0.771037   3.366800
4  1.843925  1.956811  20.901870
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-7813
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([11.270093], dtype=float32)}
{'predictions': array([3.2542899], dtype=float32)}
{'predictions': array([9.295976], dtype=float32)}
{'predictions': array([3.2169807], dtype=float32)}
{'predictions': array([20.876633], dtype=float32)}
